In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pde import CartesianGrid, solve_laplace_equation
from sfepy import data_dir

from __future__ import absolute_import
from sfepy.discrete.fem.meshio import UserMeshIO
from sfepy.mesh.mesh_generators import gen_block_mesh


In [2]:
v0 = np.array([1, 0.25])

v0.reshape(-1,1)

array([[1.  ],
       [0.25]])

## Simple tests

In [3]:
grid

CartesianGrid(bounds=((0.0, 6.283185307179586), (0.0, 6.283185307179586)), shape=(64, 64), periodic=[False, False])

In [2]:
grid = CartesianGrid([[0, 2 * np.pi]] * 3, 64)
bcs = [{"value": "sin(y)"}, {"value": "sin(x)"}, {"value":"sin(z)"}]

res = solve_laplace_equation(grid, bcs)
res.plot()


NotImplementedError: 3-dimensional Laplace matrix not implemented

## sfepy experiments

In [3]:
import numpy as nm

from sfepy.discrete.fem import Mesh, FEDomain, Field

In [5]:
mesh = Mesh.from_file(data_dir+'/meshes/2d/rectangle_tri.mesh')

sfepy:   reading mesh (/home/raul/.local/share/virtualenvs/FerroFilter-1Ki31drB/lib/python3.10/site-packages/sfepy/meshes/2d/rectangle_tri.mesh)...
sfepy:     number of vertices: 258
sfepy:     number of cells:
sfepy:       2_3: 454
sfepy:   ...done in 0.00 s


In [6]:
domain = FEDomain('domain', mesh)

In [7]:
min_x, max_x = domain.get_mesh_bounding_box()[:, 0]
eps = 1e-8 * (max_x - min_x)

omega = domain.create_region('Omega', 'all')
gamma1 = domain.create_region('Gamma1',
                              'vertices in x < %.10f' % (min_x + eps),
                              'facet')

gamma2 = domain.create_region('Gamma2',
                              'vertices in x > %.10f' % (max_x - eps),
                              'facet')

In [9]:
field = Field.from_args('fu', nm.float64, 'vector', omega,
                        approx_order=2)

In [10]:
from sfepy.discrete import (FieldVariable, Material, Integral, Function,
                            Equation, Equations, Problem)

u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

In [13]:
from sfepy.mechanics.matcoefs import stiffness_from_lame

m = Material('m', D=stiffness_from_lame(dim=2, lam=1.0, mu=1.0))
f = Material('f', val=[[0.02], [0.01]])

In [11]:
integral = Integral('i', order=3)

from sfepy.terms import Term

t1 = Term.new('dw_lin_elastic(m.D, v, u)',
              integral, omega, m=m, v=v, u=u)

t2 = Term.new('dw_volume_lvf(f.val, v)',
              integral, omega, f=f, v=v)

eq = Equation('balance', t1 + t2)
eqs = Equations([eq])

In [15]:
from sfepy.discrete.conditions import Conditions, EssentialBC

fix_u = EssentialBC('fix_u', gamma1, {'u.all' : 0.0})
def shift_u_fun(ts, coors, bc=None, problem=None, shift=0.0):
    val = shift * coors[:,1]**2
    return val

bc_fun = Function('shift_u_fun', shift_u_fun,
                  extra_args={'shift' : 0.01})
shift_u = EssentialBC('shift_u', gamma2, {'u.0' : bc_fun})

In [16]:
from sfepy.base.base import IndexedStruct
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton

ls = ScipyDirect({})

nls_status = IndexedStruct()

nls = Newton({}, lin_solver=ls, status=nls_status)

In [17]:
pb = Problem('elasticity', equations=eqs)

In [18]:
pb.save_regions_as_groups('regions')


sfepy:   saving regions as groups...
sfepy:     Omega
sfepy:     Gamma1
sfepy:     Gamma2
sfepy:   ...done


In [22]:
! sfepy-view regions.vtk -2 --grid-vector1 "[2, 0, 0]"


mesh from regions.vtk:
  points:  258
  cells:   454
  bounds:  [(-5.0, 5.0), (-10.0, 10.0), (0.0, 0.0)]
  scalars: Omega, Gamma1, Gamma2, node_groups, mat_id
  steps:   1
/home/raul/.local/share/virtualenvs/FerroFilter-1Ki31drB/lib/python3.10/site-packages/pyvista/core/pointset.py:206: PyVistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyVistaDeprecationWarning)
plot 0: Omega(step 0)
plot 1: Gamma1(step 0)
plot 2: Gamma2(step 0)


In [33]:
pb.set_bcs(ebcs=Conditions([fix_u, shift_u]))
pb.set_solver(nls)
status = IndexedStruct()
variables = pb.solve(status=status)

print('Nonlinear solver status:\n', nls_status)
print('Stationary solver status:\n', status)
pb.save_state('linear_elasticity.vtk', variables)

sfepy:   updating variables...
sfepy:   ...done
sfepy:   setting up dof connectivities...
sfepy:   ...done in 0.00 s
sfepy:   matrix shape: (1815, 1815)
sfepy:   assembling matrix graph...
sfepy:   ...done in 0.00 s
sfepy:   matrix structural nonzeros: 39145 (1.19e+00% fill)
sfepy:   updating variables...
sfepy:   ...done
sfepy:   updating materials...
sfepy:       m
sfepy:       f
sfepy:   ...done in 0.01 s
sfepy:   nls: iter: 0, residual: 1.343114e+01 (rel: 1.000000e+00)
sfepy:     residual:    0.04 [s]
sfepy:       matrix:    0.00 [s]
sfepy:        solve:    0.04 [s]
sfepy:   nls: iter: 1, residual: 2.913563e-14 (rel: 2.169260e-15)


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy:   solved in 1 steps in 0.11 seconds
Nonlinear solver status:
 IndexedStruct
  condition:
    0
  err:
    2.9135626105662025e-14
  err0:
    13.431138597204924
  ls_n_iter:
    -1
  n_iter:
    1
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']
Stationary solver status:
 IndexedStruct
  n_step:
    1
  time:
    0.10712266800010184


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

In [34]:
! sfepy-view linear_elasticity.vtk -2

mesh from linear_elasticity.vtk:
  points:  258
  cells:   454
  bounds:  [(-5.0, 5.0), (-10.0, 10.0), (0.0, 0.0)]
  scalars: node_groups, mat_id
  vectors: u
  steps:   1
/home/raul/.local/share/virtualenvs/FerroFilter-1Ki31drB/lib/python3.10/site-packages/pyvista/core/pointset.py:206: PyVistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyVistaDeprecationWarning)
Traceback (most recent call last):
  File "/home/raul/.local/share/virtualenvs/FerroFilter-1Ki31drB/bin/sfepy-view", line 8, in <module>
    sys.exit(main())
  File "/home/raul/.local/share/virtualenvs/FerroFilter-1Ki31drB/lib/python3.10/site-packages/sfepy/scripts/resview.py", line 857, in main
    plotter = pv_plot(options.filenames, options, plotter=plotter)
  File "/home/raul/.local/sh

## Magnetic field

In [ ]:
# Mesh dimensions.
dims = [0.1, 0.1]

# Mesh resolution: increase to improve accuracy.
shape = [51, 51]

def mesh_hook(mesh, mode):
    """
    Generate the block mesh.
    """
    if mode == 'read':
        mesh = gen_block_mesh(dims, shape, [0, 0], name='user_block',
                              verbose=False)
        return mesh

    elif mode == 'write':
        pass

filename_mesh = UserMeshIO(mesh_hook)

regions = {
    'Omega' : 'all',
    'Left' : ('vertices in (x < -0.0499)', 'facet'),
    'Right' : ('vertices in (x > 0.0499)', 'facet'),
    'Bottom' : ('vertices in (y < -0.0499)', 'facet'),
    'Top' : ('vertices in (y > 0.0499)', 'facet'),
    'Walls' : ('r.Left +v r.Right +v r.Bottom', 'facet'),
}

materials = {
    'fluid' : ({'viscosity' : 1.00e-2},),
}

fields = {
    'velocity': ('real', 'vector', 'Omega', 2),
    'pressure': ('real', 'scalar', 'Omega', 1),
}

variables = {
    'u' : ('unknown field', 'velocity', 0),
    'v' : ('test field', 'velocity', 'u'),
    'p' : ('unknown field', 'pressure', 1),
    'q' : ('test field', 'pressure', 'p'),
}

ebcs = {
    '1_Walls' : ('Walls', {'u.all' : 0.0}),
    '0_Driven' : ('Top', {'u.0' : 1.0, 'u.1' : 0.0}),
    'Pressure' : ('Bottom', {'p.0' : 0.0}),
}

integrals = {
    'i' : 4,
}

equations = {
    'balance' :
    """+ dw_div_grad.i.Omega(fluid.viscosity, v, u)
       + dw_convect.i.Omega(v, u)
       - dw_stokes.i.Omega(v, p) = 0""",

    'incompressibility' :
    """dw_stokes.i.Omega(u, q) = 0""",
}

solvers = {
    'ls' : ('ls.scipy_direct', {}),
    'newton' : ('nls.newton', {
        'i_max'      : 15,
        'eps_a'      : 1e-10,
        'eps_r'      : 1.0,
    }),
}

# --$\verb|blank|$--